In [13]:
# ---------------------------------------------------------------
# 0. Imports & URL
# ---------------------------------------------------------------
import pandas as pd
import numpy as np

RAW = ("https://raw.githubusercontent.com/"
       "marcbrittain/Narrabeen/master/Narrabeen_Profiles.csv")

names = ["Site", "Profile", "date", "x", "z", "Flag"]   # header row

# ---------------------------------------------------------------
# 1. Load file, keep PF1, PF4, PF8
# ---------------------------------------------------------------
df = pd.read_csv(RAW, names=names, skiprows=1, na_values=["", "NaN"])

df["date"]    = pd.to_datetime(df["date"], errors="coerce")
df["Profile"] = df["Profile"].astype(str).str.strip()           # 'PF1', 'PF4', …

keep = ["PF1", "PF4", "PF8"]
df = df[df["Profile"].isin(keep)].copy()

# nicer labels: PF1→P1, PF4→P4, PF8→P8
df["Profile"] = df["Profile"].str.replace("PF", "P", regex=False)

# ---------------------------------------------------------------
# 2. Cross-sectional volume for every survey (z ≥ 0 m)
# ---------------------------------------------------------------
def beach_vol(survey):
    active = survey[survey["z"] >= 0].sort_values("x")
    return np.trapz(active["z"], x=active["x"]) if not active.empty else np.nan

metrics = (df.groupby(["Profile", "date"])
             .apply(beach_vol)
             .reset_index(name="V"))

# ---------------------------------------------------------------
# 3. Exact volumes on four survey dates + Δ rows
# ---------------------------------------------------------------
key_dates = pd.to_datetime(
    ["2007-05-14", "2007-06-12", "2007-07-03", "2007-08-01"]
)

vol_tbl = (metrics.query("date in @key_dates")
                    .pivot(index="date", columns="Profile", values="V")
                    .reindex(key_dates))             # keep row order

dv_rows = vol_tbl.diff().iloc[1:]
dv_rows.index = [f"Δ since {d.strftime('%d %b')}" for d in dv_rows.index]

result = pd.concat([vol_tbl, dv_rows])

print("\nExact beach volumes (m³ m⁻¹) for P1, P4 & P8 and the inter-survey changes:\n")
display(result.style.format("{:,.0f}"))


/tmp/ipykernel_465/593747603.py:31: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return np.trapz(active["z"], x=active["x"]) if not active.empty else np.nan



Exact beach volumes (m³ m⁻¹) for P1, P4 & P8 and the inter-survey changes:



/tmp/ipykernel_465/593747603.py:34: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(beach_vol)


Profile,P1,P4,P8
2007-05-14 00:00:00,591,380,167
2007-06-12 00:00:00,513,284,118
2007-07-03 00:00:00,500,285,120
2007-08-01 00:00:00,510,277,119
Δ since 12 Jun,-78,-96,-49
Δ since 03 Jul,-13,1,2
Δ since 01 Aug,10,-7,-1


In [5]:
import pandas as pd
URL = "https://drive.google.com/uc?export=download&id=1V2zH5ILOBRp2PXBsHFi98NvRFJxGE4hA"
print(pd.read_csv(URL, nrows=0).columns.tolist())


['Profile ID', 'Date and time (dd/mm/yyyy HH:MM AEST)', 'Significant wave height (m)', 'Peak wave period (s)', 'Direction (degrees TN)', 'Flag']
